In [2]:
# Cell 1: Initialize EnphaseClient
import sys
sys.path.append('../src')

from enphase_client import EnphaseClient
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

print("=== ENPHASE CLIENT TEST SUITE ===\n")

# Load credentials
env_file = Path("../.env")
credentials = {}
with open(env_file, 'r') as f:
    for line in f:
        if '=' in line:
            key, value = line.strip().split('=', 1)
            credentials[key] = value

# Initialize client
client = EnphaseClient(
    access_token=credentials['ENPHASE_ACCESS_TOKEN'],
    api_key=credentials['ENPHASE_API_KEY'],
    system_id=credentials['ENPHASE_SYSTEM_ID']
)

print("EnphaseClient initialized successfully")

=== ENPHASE CLIENT TEST SUITE ===

EnphaseClient initialized successfully


In [3]:
# Cell 2: Test basic API functionality
print("=== API HEALTH AND STATUS ===\n")

# API health check
health = client.check_api_health()
if health['api_accessible']:
    print(f"API Status: Accessible")
    print(f"Current Power: {health['current_power']} W")
    print(f"Energy Today: {health['energy_today']:.1f} kWh")
    print(f"Lifetime Energy: {health['energy_lifetime']:.1f} MWh")
else:
    print(f"API Issue: {health.get('error')}")

# Current system status
status = client.get_current_status()
if status:
    print(f"\nLast Report: {status.get('last_report_at')}")
    print(f"System ID: {status.get('system_id', 'N/A')}")

=== API HEALTH AND STATUS ===

API Status: Accessible
Current Power: 0 W
Energy Today: 53.1 kWh
Lifetime Energy: 33.5 MWh

Last Report: 1758250360
System ID: 4136754


In [4]:
# Cell 3: Test energy_lifetime endpoint (main breakthrough)
print("=== HISTORICAL DATA ACCESS ===\n")

# Test recent 30 days
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

recent_data = client.get_energy_lifetime(start_date, end_date)

if not recent_data.empty:
    print(f"Recent 30 days: {len(recent_data)} days retrieved")
    print(f"Total energy: {recent_data['daily_energy_kwh'].sum():.1f} kWh")
    print(f"Average daily: {recent_data['daily_energy_kwh'].mean():.1f} kWh")

    print(f"\nLast 7 days:")
    for date, row in recent_data.tail(7).iterrows():
        print(f"  {date.strftime('%Y-%m-%d')}: {row['daily_energy_kwh']:.1f} kWh")

# Test full historical access
print(f"\n--- Full Historical Dataset ---")
full_data = client.get_historical_data()

if not full_data.empty:
    print(f"Complete history: {len(full_data)} days")
    print(f"Date range: {full_data.index.min().strftime('%Y-%m-%d')} to {full_data.index.max().strftime('%Y-%m-%d')}")
    print(f"Total lifetime: {full_data['daily_energy_kwh'].sum():.1f} kWh")
    print(f"Peak day: {full_data['daily_energy_kwh'].max():.1f} kWh")

=== HISTORICAL DATA ACCESS ===

Recent 30 days: 30 days retrieved
Total energy: 1183.1 kWh
Average daily: 39.4 kWh

Last 7 days:
  2025-09-11: 21.9 kWh
  2025-09-12: 50.9 kWh
  2025-09-13: 50.7 kWh
  2025-09-14: 26.6 kWh
  2025-09-15: 23.6 kWh
  2025-09-16: 16.7 kWh
  2025-09-17: 33.8 kWh

--- Full Historical Dataset ---
Complete history: 889 days
Date range: 2023-04-13 to 2025-09-17
Total lifetime: 33499.2 kWh
Peak day: 61.7 kWh


In [5]:
# Cell 4: Data analysis and seasonal patterns
print("=== DATA ANALYSIS ===\n")

if 'full_data' in locals() and not full_data.empty:
    # Monthly analysis
    full_data['month'] = full_data.index.month
    monthly_stats = full_data.groupby('month')['daily_energy_kwh'].agg(['mean', 'max', 'count'])

    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    print("Monthly Production Statistics:")
    for month_num, stats in monthly_stats.iterrows():
        month_name = months[month_num-1]
        print(f"  {month_name}: Avg={stats['mean']:.1f} kWh/day, Peak={stats['max']:.1f} kWh, Days={int(stats['count'])}")

    # Peak and low analysis
    peak_month = monthly_stats['mean'].idxmax()
    low_month = monthly_stats['mean'].idxmin()

    print(f"\nSeasonal Summary:")
    print(f"  Peak month: {months[peak_month-1]} ({monthly_stats.loc[peak_month, 'mean']:.1f} kWh/day)")
    print(f"  Low month: {months[low_month-1]} ({monthly_stats.loc[low_month, 'mean']:.1f} kWh/day)")

    # Recent performance
    last_30_days = full_data.tail(30)
    recent_avg = last_30_days['daily_energy_kwh'].mean()
    print(f"  Recent 30-day average: {recent_avg:.1f} kWh/day")

=== DATA ANALYSIS ===

Monthly Production Statistics:
  Jan: Avg=30.4 kWh/day, Peak=45.0 kWh, Days=62
  Feb: Avg=32.8 kWh/day, Peak=50.9 kWh, Days=57
  Mar: Avg=38.8 kWh/day, Peak=58.3 kWh, Days=62
  Apr: Avg=33.6 kWh/day, Peak=59.9 kWh, Days=78
  May: Avg=40.4 kWh/day, Peak=61.7 kWh, Days=93
  Jun: Avg=45.9 kWh/day, Peak=59.2 kWh, Days=90
  Jul: Avg=42.9 kWh/day, Peak=58.2 kWh, Days=93
  Aug: Avg=40.8 kWh/day, Peak=57.5 kWh, Days=93
  Sep: Avg=37.8 kWh/day, Peak=57.9 kWh, Days=77
  Oct: Avg=40.8 kWh/day, Peak=53.4 kWh, Days=62
  Nov: Avg=31.7 kWh/day, Peak=48.9 kWh, Days=60
  Dec: Avg=27.6 kWh/day, Peak=41.2 kWh, Days=62

Seasonal Summary:
  Peak month: Jun (45.9 kWh/day)
  Low month: Dec (27.6 kWh/day)
  Recent 30-day average: 39.4 kWh/day


In [6]:
# Cell 5: Data export and integration capabilities
print("=== DATA EXPORT AND INTEGRATION ===\n")

# Export capability test
export_filename = "../data/processed/api_historical_data.csv"
export_success = client.export_to_csv(
    filename=export_filename,
    start_date=datetime(2024, 1, 1),  # Last year's data
    end_date=datetime.now()
)

if export_success:
    print(f"Data exported successfully to {export_filename}")

    # Verify export
    exported_data = pd.read_csv(export_filename, index_col=0, parse_dates=True)
    print(f"Exported {len(exported_data)} days of data")

# Integration with ML models
print(f"\n--- ML Model Integration ---")
print("Data format compatible with existing ML pipeline:")
print("- Daily production totals (kWh)")
print("- DateTime index for time series analysis")
print("- Ready for weather data merging")
print("- Can replace CSV data source entirely")

print(f"\nNext steps:")
print("- Integrate with weather-enhanced ML models")
print("- Set up automated daily data collection")
print("- Build real-time monitoring dashboard")

=== DATA EXPORT AND INTEGRATION ===

Exported 626 days of data to ../data/processed/api_historical_data.csv
Data exported successfully to ../data/processed/api_historical_data.csv
Exported 626 days of data

--- ML Model Integration ---
Data format compatible with existing ML pipeline:
- Daily production totals (kWh)
- DateTime index for time series analysis
- Ready for weather data merging
- Can replace CSV data source entirely

Next steps:
- Integrate with weather-enhanced ML models
- Set up automated daily data collection
- Build real-time monitoring dashboard
